# Scaling Analysis: Runtime vs. Problem Size

This notebook empirically measures how algorithm performance scales with problem size ($N$).
We use the **TSP** problem and **NSGA-II** algorithm as a case study.

## Goals
1.  Measure runtime for a fixed number of evaluations as $N$ increases.
2.  Fit a complexity curve ($O(N)$, $O(N^2)$) to the data.
3.  (Optional) discuss theoretical parallel speedups.
4.  **NEW**: Measure **Convergence Scaling** (Evaluations to target quality vs $N$).

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

# VAMOS imports
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.problems import TSP
from vamos import make_problem_selection
from vamos.foundation.metrics.hypervolume import compute_hypervolume

## 1. Experiment Setup
We define a function to run a quick benchmark for a given number of cities.

In [ ]:
def benchmark_tsp_scaling(n_cities_list, n_evals=1000, pop_size=100):
    results = []

    print(f"Benchmarking NSGA-II (Pop={pop_size}, Evals={n_evals}) on TSP scaling...")

    for n_cities in n_cities_list:
        # Create random TSP instance on a circle (predictable structure)
        problem = TSP(n_cities=n_cities)

        # Configure NSGA-II
        algo_config = (
            NSGAIIConfig.builder()
            .pop_size(pop_size)
            .crossover("pmx", prob=0.9)
            .mutation("swap", prob=0.1)
              # Ensure we test the NumPy path
            .build()
        )

        # Measure Time
        start_time = time.time()
        optimize(
            problem=problem, algorithm="nsgaii", algorithm_config=algo_config, termination=("n_eval", n_evals), seed=42
        )
        duration = time.time() - start_time

        print(f"N={n_cities}: {duration:.4f}s")
        results.append({"N": n_cities, "Time": duration})

    return pd.DataFrame(results)


# Define range of N
# We go up to 200 for interactivity. Increase to 500+ for serious checks.
n_range = [20, 40, 60, 80, 100, 150, 200, 300]
df_res = benchmark_tsp_scaling(n_range, n_evals=2000, pop_size=50)

## 2. Visualize Scaling & Curve Fitting
We fit a polynomial $T(N) = a N^2 + b N + c$ (since non-dominated sort is usually $O(M N^2)$ or $O(N^2)$ depending on implementation, and distance matrix calc for TSP is $O(N^2)$).
Wait, for TSP, the genome length is $N$. 
- Variation operators are typically $O(N)$.
- Evaluation (tour length) is $O(N)$.
- So overall per-generation cost should be roughly linear $O(N)$ * Population, unless DOMINATION sort dominates.
- Let's see empirically.

In [ ]:
def quad_func(x, a, b, c):
    return a * x**2 + b * x + c


def lin_func(x, a, b):
    return a * x + b


x_data = df_res["N"].values
y_data = df_res["Time"].values

# Fit
popt_q, _ = curve_fit(quad_func, x_data, y_data)
popt_l, _ = curve_fit(lin_func, x_data, y_data)

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_res, x="N", y="Time", s=100, label="Observed")

x_plot = np.linspace(min(x_data), max(x_data), 100)
plt.plot(x_plot, quad_func(x_plot, *popt_q), "r--", label=f"Quadratic Fit ($N^2$)")
plt.plot(x_plot, lin_func(x_plot, *popt_l), "g:", label=f"Linear Fit ($N$)")

plt.title("Runtime Scaling vs Problem Size (TSP + NSGA-II)")
plt.xlabel("Number of Cities (N)")
plt.ylabel("Runtime (s)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Theoretical Parallel Speedup
Assuming evaluating the population takes a fraction $P$ of the total time (typically high for complex simulations, lower for simple TSP).
Amdahl's Law: $S(n) = \frac{1}{(1-P) + \frac{P}{n}}$

In [ ]:
def plot_amdahl(p_parallel_fractions=[0.5, 0.75, 0.9, 0.99]):
    cores = np.arange(1, 33)
    plt.figure(figsize=(10, 6))

    for p in p_parallel_fractions:
        speedup = 1 / ((1 - p) + (p / cores))
        plt.plot(cores, speedup, label=f"P={p * 100:.0f}% Parallelizable")

    plt.plot(cores, cores, "k--", alpha=0.3, label="Ideal Linear Speedup")
    plt.title("Theoretical Amdahl's Law Speedup")
    plt.xlabel("Number of Cores")
    plt.ylabel("Speedup Factor")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


plot_amdahl()

---

## 4. Convergence Scaling: Search Difficulty vs Problem Size

The previous sections measure **implementation speed** (how fast the code runs).
This section measures **algorithmic efficiency** (how many evaluations to find a good solution).

We ask: *"How many evaluations does NSGA-II need to reach 95% of the best-achievable Hypervolume?"* as problem size $N$ increases.

**Setup**: We use **ZDT1** (continuous, scalable in $N_{var}$) for this analysis since it has a known optimal HV.

In [ ]:
from vamos.problems import ZDT1


def run_with_hv_tracking(n_var, max_evals=10000, pop_size=56, offspring_size=14, seed=42):
    """
    Run NSGA-II on ZDT1 and track HV after each generation.
    Returns a list of (eval_count, hv) tuples.
    """
    problem = ZDT1(n_var=n_var)
    ref_point = np.array([1.1, 1.1])

    hv_history = []

    # We simulate generations by running with increasing budgets
    # A more elegant approach would use a callback, but this works for a notebook demo.
    gen_size = offspring_size
    for budget in range(gen_size, max_evals + 1, gen_size):
        cfg = (
            NSGAIIConfig.builder()
            .pop_size(pop_size)
            .offspring_size(offspring_size)
            .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
            .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
            .selection("tournament", pressure=9)
            .repair("round")
            .archive(size=56).archive_type("hypervolume")
            
            .build()
        )
        res = optimize(**dict(problem=problem, algorithm="nsgaii", algorithm_config=cfg, termination=("n_eval", budget), seed=seed))
        if res.F is not None and len(res.F) > 0:
            hv = compute_hypervolume(res.F, ref_point)
        else:
            hv = 0.0
        hv_history.append((budget, hv))

        # Early stop if we seem converged (HV not changing much)
        if len(hv_history) > 3 and abs(hv_history[-1][1] - hv_history[-3][1]) < 1e-4:
            break

    return hv_history


# Quick test
print("Testing HV tracking on ZDT1 (n_var=10)...")
test_history = run_with_hv_tracking(n_var=10, max_evals=1000)
print(f"Final HV: {test_history[-1][1]:.4f} after {test_history[-1][0]} evals")

In [ ]:
def find_evals_to_target(hv_history, target_fraction=0.95):
    """
    Find the first evaluation count where HV >= target_fraction * final_hv.
    """
    if not hv_history:
        return np.nan
    final_hv = hv_history[-1][1]
    if final_hv <= 0:
        return np.nan
    target_hv = target_fraction * final_hv

    for evals, hv in hv_history:
        if hv >= target_hv:
            return evals
    return hv_history[-1][0]  # Never reached, return max


# Run for multiple n_var values
n_var_list = [10, 20, 30, 50]
convergence_results = []

print("Running Convergence Scaling Experiment...")
for n_var in n_var_list:
    print(f"  n_var={n_var}")
    history = run_with_hv_tracking(n_var=n_var, max_evals=5000, pop_size=50, seed=42)
    evals_95 = find_evals_to_target(history, 0.95)
    final_hv = history[-1][1]
    convergence_results.append({"n_var": n_var, "Evals_95": evals_95, "Final_HV": final_hv})
    print(f"    Evals to 95%: {evals_95}, Final HV: {final_hv:.4f}")

df_conv = pd.DataFrame(convergence_results)
df_conv

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_conv, x="n_var", y="Evals_95", palette="viridis")
plt.title("Convergence Scaling: Evaluations to 95% HV vs Problem Size")
plt.xlabel("Number of Decision Variables ($N_{var}$)")
plt.ylabel("Evaluations to Reach 95% of Final HV")
plt.grid(True, alpha=0.3, axis="y")
plt.show()

# Fit a line to see if it's linear or super-linear
try:
    popt_conv, _ = curve_fit(lin_func, df_conv["n_var"].values, df_conv["Evals_95"].values)
    print(f"Linear Fit: Evals = {popt_conv[0]:.2f} * N + {popt_conv[1]:.2f}")
except:
    print("Could not fit linear model.")